In [1]:
library(caret, quiet=TRUE);
library(base64enc)
library(httr, quiet=TRUE)



Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:
set.seed(1960)

create_model  =  function() {

    model  <- preProcess(iris[, -5], method = c("center", "scale"))
    
    return(model)
}

In [3]:
# dataset
model = create_model()

In [4]:
pred <- predict(model, as.matrix(iris[, -5]))
head(pred)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
-0.8976739,1.01560199,-1.335752,-1.311052
-1.1392005,-0.13153881,-1.335752,-1.311052
-1.3807271,0.32731751,-1.392399,-1.311052
-1.5014904,0.09788935,-1.279104,-1.311052
-1.0184372,1.24503015,-1.335752,-1.311052
-0.5353840,1.93331463,-1.165809,-1.048667


In [5]:
print(model)

Created from 150 samples and 4 variables

Pre-processing:
  - centered (4)
  - ignored (0)
  - scaled (4)



In [6]:
cbind(model$mean , model$std)

Sepal.Length,5.843333,0.8280661
Sepal.Width,3.057333,0.4358663
Petal.Length,3.758000,1.7652982
Petal.Width,1.199333,0.7622377


In [7]:
model$method

$center
[1] "Sepal.Length" "Sepal.Width"  "Petal.Length" "Petal.Width" 

$scale
[1] "Sepal.Length" "Sepal.Width"  "Petal.Length" "Petal.Width" 

$ignore
character(0)

# SQL Code Generation

In [8]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "caret_rpart_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [9]:
lModelSQL = test_ws_sql_gen(model)

In [10]:
cat(lModelSQL)


SELECT "ADS"."KEY" AS "KEY", (CAST("ADS"."Feature_0" AS FLOAT) - 5.843333333333334) / 0.8280661279778629 AS "Feature_0", (CAST("ADS"."Feature_1" AS FLOAT) - 3.0573333333333332) / 0.4358662849366982 AS "Feature_1", (CAST("ADS"."Feature_2" AS FLOAT) - 3.758) / 1.7652982332594664 AS "Feature_2", (CAST("ADS"."Feature_3" AS FLOAT) - 1.199333333333333) / 0.7622376689603465 AS "Feature_3" 
FROM "INPUT_DATA" AS "ADS"

# Execute the SQL Code

In [11]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [12]:
dataset = iris[,-5]

df_sql = as.data.frame(dataset)
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

head(df_sql)

Feature_0,Feature_1,Feature_2,Feature_3,KEY
5.1,3.5,1.4,0.2,1
4.9,3.0,1.4,0.2,2
4.7,3.2,1.3,0.2,3
4.6,3.1,1.5,0.2,4
5.0,3.6,1.4,0.2,5
5.4,3.9,1.7,0.4,6


In [13]:
# colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

In [14]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out)

KEY,Feature_0,Feature_1,Feature_2,Feature_3
1,-0.8976739,1.01560199,-1.335752,-1.311052
2,-1.1392005,-0.13153881,-1.335752,-1.311052
3,-1.3807271,0.32731751,-1.392399,-1.311052
4,-1.5014904,0.09788935,-1.279104,-1.311052
5,-1.0184372,1.24503015,-1.335752,-1.311052
6,-0.5353840,1.93331463,-1.165809,-1.048667


# R Preprocess Output

In [15]:
preprocessed  =  predict(model, iris[,-5])
df_r_out = data.frame(preprocessed)
names(df_r_out) = sprintf("Feature_%d",0:(ncol(df_r_out) - 1))

df_r_out$KEY = seq.int(nrow(dataset))

head(df_r_out)



Feature_0,Feature_1,Feature_2,Feature_3,KEY
-0.8976739,1.01560199,-1.335752,-1.311052,1
-1.1392005,-0.13153881,-1.335752,-1.311052,2
-1.3807271,0.32731751,-1.392399,-1.311052,3
-1.5014904,0.09788935,-1.279104,-1.311052,4
-1.0184372,1.24503015,-1.335752,-1.311052,5
-0.5353840,1.93331463,-1.165809,-1.048667,6


# Compare R and SQL output

In [21]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_R","_SQL"))
head(df_merge)

KEY,Feature_0_R,Feature_1_R,Feature_2_R,Feature_3_R,Feature_0_SQL,Feature_1_SQL,Feature_2_SQL,Feature_3_SQL
1,-0.8976739,1.01560199,-1.335752,-1.311052,-0.8976739,1.01560199,-1.335752,-1.311052
2,-1.1392005,-0.13153881,-1.335752,-1.311052,-1.1392005,-0.13153881,-1.335752,-1.311052
3,-1.3807271,0.32731751,-1.392399,-1.311052,-1.3807271,0.32731751,-1.392399,-1.311052
4,-1.5014904,0.09788935,-1.279104,-1.311052,-1.5014904,0.09788935,-1.279104,-1.311052
5,-1.0184372,1.24503015,-1.335752,-1.311052,-1.0184372,1.24503015,-1.335752,-1.311052
6,-0.5353840,1.93331463,-1.165809,-1.048667,-0.5353840,1.93331463,-1.165809,-1.048667


In [22]:
diffs_df = df_merge[df_merge$Feature_1_R != df_merge$Feature_1_SQL,]
head(diffs_df)

KEY,Feature_0_R,Feature_1_R,Feature_2_R,Feature_3_R,Feature_0_SQL,Feature_1_SQL,Feature_2_SQL,Feature_3_SQL


In [18]:
stopifnot(nrow(diffs_df) == 0)

In [19]:
summary(df_sql_out)

      KEY           Feature_0          Feature_1         Feature_2      
 Min.   :  1.00   Min.   :-1.86378   Min.   :-2.4258   Min.   :-1.5623  
 1st Qu.: 38.25   1st Qu.:-0.89767   1st Qu.:-0.5904   1st Qu.:-1.2225  
 Median : 75.50   Median :-0.05233   Median :-0.1315   Median : 0.3354  
 Mean   : 75.50   Mean   : 0.00000   Mean   : 0.0000   Mean   : 0.0000  
 3rd Qu.:112.75   3rd Qu.: 0.67225   3rd Qu.: 0.5567   3rd Qu.: 0.7602  
 Max.   :150.00   Max.   : 2.48370   Max.   : 3.0805   Max.   : 1.7799  
   Feature_3      
 Min.   :-1.4422  
 1st Qu.:-1.1799  
 Median : 0.1321  
 Mean   : 0.0000  
 3rd Qu.: 0.7880  
 Max.   : 1.7064  

In [20]:
summary(df_r_out)

   Feature_0          Feature_1         Feature_2         Feature_3      
 Min.   :-1.86378   Min.   :-2.4258   Min.   :-1.5623   Min.   :-1.4422  
 1st Qu.:-0.89767   1st Qu.:-0.5904   1st Qu.:-1.2225   1st Qu.:-1.1799  
 Median :-0.05233   Median :-0.1315   Median : 0.3354   Median : 0.1321  
 Mean   : 0.00000   Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.0000  
 3rd Qu.: 0.67225   3rd Qu.: 0.5567   3rd Qu.: 0.7602   3rd Qu.: 0.7880  
 Max.   : 2.48370   Max.   : 3.0805   Max.   : 1.7799   Max.   : 1.7064  
      KEY        
 Min.   :  1.00  
 1st Qu.: 38.25  
 Median : 75.50  
 Mean   : 75.50  
 3rd Qu.:112.75  
 Max.   :150.00  